# Labware Calibration
To use a new piece of labware, we need to know it's well count, spacing, and height. Run through this calibration process for each new labware you wish to use; the results are saved in config/labware and can be reused in subsequent notebooks.

In [2]:
from duckbot.Machine import Machine, get_root_dir
import json
import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
m = Machine()

configuring tools


In [4]:
m.set_plate(config="base_plate_config")

In [6]:
m.plate.slots

{0: {'origin': [14.5, 91.2], 'labware': None},
 1: {'origin': [15.1, 187.8], 'labware': None},
 2: {'origin': [15.9, 284.7], 'labware': None},
 3: {'origin': [155.8, 283.6], 'labware': None},
 4: {'origin': [155.1, 186.7], 'labware': None},
 5: {'origin': [154.3, 89.8], 'labware': None}}

In [7]:
# what is the name of this labware?
# this will be how you refer to this labware in all future experiments
# anything works; we suggest the naming convention <brand>_<wellcount>_<description>
labware_name = "opentrons_96_tiprack_300ul"
calibration_positions = {}

In [8]:
# how many wells, rows, and columns does this labware have?
# note: rows are generally letters, columns are numbers
well_count = 96
row_count = 8
column_count = 12

In [9]:
m._configured_tools

{0: 'Inoculation Loop',
 1: 'BrokenTrons',
 2: 'Side Camera',
 3: 'Top-Down Camera'}

In [10]:
# Pick up your camera tool
m.tool_change(3)

In [11]:
# drop the bed down and load your labware 
m.move_to(z=100)

In [14]:
# move to the slot 0's origin
slot_index = 0
origin_x = m.plate.slots[slot_index]['origin'][0]
origin_y = m.plate.slots[slot_index]['origin'][1]

m.move_to(x=origin_x, y=origin_y)

In [13]:
# using duet web control, *carefully* move the bed up until the camera is just above the labware
# then move down by the camera's focal length
# ToDo: set camera focal length in config
m.move(dz=30)

In [15]:
# open a camera feed and center the camera over well A1
m.tool.video_stream(2)

(320, 240)


In [19]:
# save this position
a1_x = float(m.get_position()['X'])
a1_y = float(m.get_position()['Y'])

calibration_positions["A1"] = (a1_x - origin_x, a1_y - origin_y)

In [20]:
# repeat for the last well in row A
m.tool.video_stream(2)

(320, 240)


In [23]:
# save this position
aj_x = float(m.get_position()['X'])
aj_y = float(m.get_position()['Y'])

calibration_positions[f"A{column_count}"] = (aj_x - origin_x, aj_y - origin_y)

In [24]:
# repeat for the well in the last row and column
m.tool.video_stream(2)

(320, 240)


In [29]:
ij_x = float(m.get_position()['X'])
ij_y = float(m.get_position()['Y'])


letter = chr(ord('@')+row_count) # this converts a number to a letter
calibration_positions[f"{letter}{column_count}"] = (ij_x - origin_x, ij_y - origin_y)

In [30]:
labware_config_path = os.path.join(get_root_dir(), f'config/labware/{labware_name}.json')
labware_config = {}
labware_config['calibration_positions'] = calibration_positions
labware_config['well_count'] = well_count
labware_config['row_count'] = row_count
labware_config['column_count'] = column_count
with open(labware_config_path, 'w') as f:
    json.dump(labware_config, f, indent=4)